In [1]:
import torch
import torch.nn.functional as F
from spherical_distortion.functional import create_tangent_images, unresample
from spherical_distortion.util import *
import matplotlib.pyplot as plt
from skimage import io
import os

import numpy as np
import _spherical_distortion_ext._mesh as _mesh
import argparse
sys.path.append('/home/imonalc/Keypoints/')
from utils.eqr2cub import *
from utils.cub2eqr import *

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_theta_torch(x, y):
    theta = torch.where(y < 0, (-1) * torch.atan2(y, x), 2 * math.pi - torch.atan2(y, x))
    return theta

In [122]:
def create_equirectangler_to_bottom_and_top_map(input_w, input_h, output_sqr, z):
    x, y = torch.meshgrid(torch.linspace(-output_sqr/2.0, output_sqr/2.0-1, output_sqr), 
                          torch.linspace(-output_sqr/2.0, output_sqr/2.0-1, output_sqr),indexing='ij')
    x, y = x.to(device), y.to(device)
    z = torch.tensor(z).to(device)

    
    rho = torch.sqrt(x**2 + y**2 + z**2)
    norm_theta = get_theta_torch(x, y) / (2 * math.pi)
    print(norm_theta)
    norm_phi = (math.pi - torch.acos(z / rho)) / math.pi

    ix = norm_theta * input_w
    iy = norm_phi * input_h

    # Boundary handling
    ix = torch.where(ix >= input_w, ix - input_w, ix)
    iy = torch.where(iy >= input_h, iy - input_h, iy)
    
    return ix.cpu().numpy(), iy.cpu().numpy()

In [123]:
def create_bottom_and_top_to_equirectangular_map(input_w, input_h, output_sqr, iz):
    # Create a grid for the bottom map
    ix, iy = torch.meshgrid(torch.linspace(-input_h/2, input_h/2-1, input_h),
                          torch.linspace(-input_w/2, input_w/2-1, input_w), indexing='ij')
    ix, iy = ix.to(device), iy.to(device)
    iz = torch.tensor(iz).to(device)
    
    # Convert the Cartesian coordinates to spherical coordinates
    rho = torch.sqrt(ix**2 + iy**2 + iz**2)
    theta = get_theta_torch(x, y)
    phi = torch.acos(z / rho)
    
    # Convert the spherical coordinates to equirectangular coordinates
    norm_theta = theta / (2 * math.pi)
    norm_phi = phi / math.pi
    ix = norm_theta * input_w
    iy = (1.0 - norm_phi) * input_h  # invert the y-axis since the image coordinate system is top-down
    
    # Boundary handling
    ix = torch.where(ix >= input_w, ix - input_w, ix)
    iy = torch.where(iy >= input_h, iy - input_h, iy)
    
    return ix.cpu().numpy(), iy.cpu().numpy()

In [124]:
# Sample usage:
top_img = torch.tensor(cv2.imread('output/top.png'))
front_img = torch.tensor(cv2.imread('output/front.png'))
left_img = torch.tensor(cv2.imread('output/left.png'))
right_img = torch.tensor(cv2.imread('output/right.png'))
back_img = torch.tensor(cv2.imread('output/back.png'))
bottom_img = torch.tensor(cv2.imread('output/bottom.png'))
cubemaps = [top_img, front_img, left_img, right_img, back_img, bottom_img]

In [125]:
img = cv2.imread("O.jpg")
input_h, input_w, _ = img.shape
output_sqr = int(input_w / 4)
normalized_f = 1

In [126]:
z = (output_sqr / (2.0 * normalized_f))
bottom_map_x, bottom_map_y = create_equirectangler_to_bottom_and_top_map(input_w, input_h, output_sqr, z)
bottom_img = cv2.remap(img, bottom_map_x.astype("float32"), bottom_map_y.astype("float32"), cv2.INTER_CUBIC)
cv2.imwrite("aaa.jpg", bottom_img)

tensor([[831.3844, 830.8074, 830.2313,  ..., 829.6559, 830.2313, 830.8074],
        [830.8074, 830.2301, 829.6536,  ..., 829.0778, 829.6536, 830.2301],
        [830.2313, 829.6536, 829.0766,  ..., 828.5004, 829.0766, 829.6536],
        ...,
        [829.6559, 829.0778, 828.5004,  ..., 827.9239, 828.5004, 829.0778],
        [830.2313, 829.6536, 829.0766,  ..., 828.5004, 829.0766, 829.6536],
        [830.8074, 830.2301, 829.6536,  ..., 829.0778, 829.6536, 830.2301]],
       device='cuda:0') torch.Size([960, 960])


True

In [104]:
print(bottom_map_x.shape, bottom_map_y.shape)
print(bottom_map_x)

(960, 960) (960, 960)
[[1440.      1440.6373  1441.2759  ... 2398.0842  2398.7239  2399.3625 ]
 [1439.3625  1440.      1440.6385  ... 2398.7214  2399.3613  2400.     ]
 [1438.7241  1439.3613  1440.      ... 2399.36    2400.      2400.6387 ]
 ...
 [ 481.91586  481.2786   480.63995 ... 3360.      3359.36    3358.7212 ]
 [ 481.2759   480.63864  480.      ... 3360.64    3360.      3359.361  ]
 [ 480.63733  480.       479.36133 ... 3361.2786  3360.6387  3360.     ]]


In [105]:
# Test the inverse transformation function
input_w, input_h, output_sqr = 2048, 1024, 512  # example values
z = (output_sqr / (2.0 * 1))  # assuming normalized_f = 1
inverse_map_x, inverse_map_y = create_bottom_and_top_to_equirectangular_map(input_w, input_h, output_sqr, z)
inverse_map_x, inverse_map_y

(array([[ 663.12555,  663.253  ,  663.3806 , ..., 1384.4916 , 1384.6194 ,
         1384.747  ],
        [ 662.87085,  662.9981 ,  663.12555, ..., 1384.7467 , 1384.8744 ,
         1385.0018 ],
        [ 662.6159 ,  662.74304,  662.87036, ..., 1385.002  , 1385.1296 ,
         1385.257  ],
        ...,
        [ 361.63922,  361.51224,  361.38507, ..., 1686.7423 , 1686.6149 ,
         1686.4878 ],
        [ 361.38406,  361.25696,  361.1296 , ..., 1686.9979 , 1686.8705 ,
         1686.7429 ],
        [ 361.12912,  361.00186,  360.8744 , ..., 1687.2533 , 1687.1256 ,
         1686.9982 ]], dtype=float32),
 array([[583.70496, 583.75916, 583.81354, ..., 583.8679 , 583.81354,
         583.75916],
        [583.73206, 583.7864 , 583.8408 , ..., 583.89526, 583.8408 ,
         583.7864 ],
        [583.7591 , 583.8135 , 583.868  , ..., 583.9225 , 583.868  ,
         583.8135 ],
        ...,
        [583.7862 , 583.84064, 583.89514, ..., 583.9497 , 583.89514,
         583.84064],
        [583.7591 , 5

In [106]:
eq_img = np.zeros((3840, 1920, 3))
eq_img = cv2.remap(bottom_img, inverse_map_x.astype("float32"), inverse_map_y.astype("float32"), cv2.INTER_CUBIC)
cv2.imwrite("eq.jpg", eq_img)

True